# Init

In [ ]:
import glob
from PIL import Image, ImageEnhance, ImageDraw, ImageFilter
import matplotlib.pyplot as plt
import numpy as np
import PIL
import torch
from sklearn import preprocessing
import itertools
import random
import os
import torch.nn.functional as F
from torchmetrics.functional import accuracy
import pytorch_lightning as pl
import torchmetrics
from torchvision.transforms import ToTensor, RandomErasing, Compose, ToPILImage
from torch import nn, Tensor
from torch.utils.data import random_split, Dataset ,DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torchvision.models import vgg16, resnet18
from typing import List 
from sklearn.metrics import classification_report
import time

import pandas as pd

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy import reshape
import seaborn as sns

## Pretext Task

Addestrare un modello per generare le rappresentazioni (attention maps) rispetto alle immagini anomale e normali

Modello: Res-Net18

### Funzioni

In [ ]:
import math

def generate_rotations(image:Image):
  r90 = image.rotate(90)
  r180 = image.rotate(180)
  r270 = image.rotate(270)
  return image, r90, r180, r270


def generate_patch(image:Image, 
                    area_ratio=(0.02, 0.15), 
                    aspect_ratio=((0.3, 1),(1, 3.3))):
  #print('generate_patch', area_ratio)
  img_area = image.size[0] * image.size[1]
  patch_area = random.uniform(area_ratio[0], area_ratio[1]) * img_area
  patch_aspect = random.choice([random.uniform(*aspect_ratio[0]), random.uniform(*aspect_ratio[1])])
  patch_w  = int(np.sqrt(patch_area*patch_aspect))
  patch_h = int(np.sqrt(patch_area/patch_aspect))
  org_w, org_h = image.size

  patch_left, patch_top = random.randint(0, org_w - patch_w), random.randint(0, org_h - patch_h)
  patch_right, patch_bottom = patch_left + patch_w, patch_top + patch_h
  paste_left, paste_top = random.randint(0, org_w - patch_w), random.randint(0, org_h - patch_h)
  
  return image.crop((patch_left, patch_top, patch_right, patch_bottom)), (paste_left, paste_top)


def paste_patch(image, patch, coords, mask = None):
  aug_image = image.copy()
  aug_image.paste(patch, (coords[0], coords[1]), mask=mask)
  return aug_image


def apply_patch_augmentations(patch:Image, 
                              augmentations:transforms.ColorJitter=None):
  patch = patch.filter(ImageFilter.GaussianBlur(random.randint(0, 5)))
  return augmentations(patch)


def random_color():
  return random.randint(10,240)

def generate_scar(imsize:tuple, w_range=(2,16), h_range=(10,25)):
  img_w, img_h = imsize

  #dimensioni sezione
  scar_w = random.randint(w_range[0], w_range[1])
  scar_h = random.randint(h_range[0], h_range[1])

  r = random_color()
  g = random_color()
  b = random_color()

  color = (r,g,b)

  scar = Image.new('RGBA', (scar_w, scar_h), color=color)
  angle = random.randint(-45, 45)
  scar = scar.rotate(angle, expand=True)

  #posizione casuale della sezione
  left, top = random.randint(0, img_w - scar_w), random.randint(0, img_h - scar_h)
  return scar, (left, top)

In [ ]:
def get_impaths(main_path):
  return sorted([f for f in glob.glob(main_path+'*.png', recursive = True)])

def load_imgs(main_path, imsize):
  filenames = get_impaths(main_path)
  images = []
  for impath in filenames:
    x = Image.open(impath)
    x = x.resize(imsize)
    images.append(x)
  return images

def generate_dataset(main_path:str, 
                      imsize=(256,256), 
                      offset=0.3,
                      area_ratio_patch=(0.02, 0.15),
                      aspect_ratio_patch=((0.3, 1),(1, 3.3)),
                      scar_width=(2,16),
                      scar_thiccness=(10,25)):

  #raw_images = load_imgs(main_path, imsize)
  raw_images_filenames = get_impaths(main_path)
  data = []
  labels = []
  #print('generate_dataset', area_ratio_patch)
  augs = transforms.ColorJitter(brightness = offset,
                              contrast = offset,
                              saturation = offset,
                              hue = offset)

  for image_filename in raw_images_filenames:
    image = Image.open(image_filename).resize(imsize)

    r0, r90, r180, r270 = generate_rotations(image)
    rotations = [r0, r90, r180, r270]

    for img in rotations:
      #img normale
      data.append(img)
      labels.append(0)

      data.append(augs(img))
      labels.append(0)
      
      #cutpaste
      x, coords = generate_patch(img, area_ratio_patch, aspect_ratio_patch)
      x = apply_patch_augmentations(x, augs)
      new_img = paste_patch(img, x, coords)
      data.append(new_img)
      labels.append(1)
      
      #scar
      x, coords = generate_scar(img.size, scar_width, scar_thiccness)
      new_img = paste_patch(img, x, coords, x)
      data.append(new_img)
      labels.append(1)
  return data, labels 

In [ ]:
def load_mvtec_dataset_imgs(main_path, imsize=(256,256)):
  data = []
  labels = []
  classes = [name for name in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, name))]
  for data_class in classes:
    if data_class=='good':
      raw_images_filenames = get_impaths(main_path+data_class+'/')
      for image_filename in raw_images_filenames:
        data.append(Image.open(image_filename).resize(imsize))
        labels.append(0)
    else:
      raw_images_filenames = get_impaths(main_path+data_class+'/')
      for image_filename in raw_images_filenames:
        data.append(Image.open(image_filename).resize(imsize))
        labels.append(1)
  return data, labels

In [ ]:
def list2np(images, labels):
  x = np.array([np.array(a, dtype=np.uint8) for a in images])
  y = np.array(labels, dtype=int)
  return x,y

def np2tensor(images, labels):
  images = torch.as_tensor(images, dtype=torch.uint8)
  labels = torch.as_tensor(labels, dtype=int)
  return images,labels

### Dataset

In [ ]:
class PretextTaskDataset(Dataset):
  def __init__(self, images, labels, transform=None, target_transform=None):
    self.transform = transform
    self.target_transform = target_transform
    self.images = images
    self.labels = labels
    
  def __len__(self):
        return len(self.labels)
  
  def __getitem__(self, index):
    image = self.images[index]
    label = self.labels[index]
    if self.transform:
      image = torch.permute(image, (2, 1, 0))
      image = self.transform(image)
    if self.target_transform:
      label = self.target_transform(label)
    return image, int(label)

In [ ]:
class PretextTaskDatamodule(pl.LightningDataModule):
  def __init__(self, 
                main_path:str, 
                batch_size:int=64, 
                seed:int=0, 
                imsize:int=(256,256),
                train_test_split:float=0.33,
                train_val_split:float=0.22,
                area_ratio_patch = (0.02, 0.15),
                aspect_ratio_patch = ((0.3, 1),(1, 3.3)),
                scar_width = (2,16),
                scar_thiccness = (10,25),
                augmentation_offset = 0.1):
    #print('PretextTaskDatamodule.init',area_ratio_patch)
    self.save_hyperparameters()
    self.prepare_data_per_node = False
    self.main_path = main_path
    self.batch_size = batch_size
    self.test_size = train_test_split
    self.val_size = train_val_split
    self.imsize = imsize
    self.seed = seed
    

    self.area_ratio_patch = area_ratio_patch
    self.aspect_ratio_patch = aspect_ratio_patch
    self.scar_width = scar_width
    self.scar_thiccness = scar_thiccness
    self.augmentation_offset = augmentation_offset
    
    random.seed(seed)
    np.random.seed(seed)
    torch.random.manual_seed(seed)

    self.transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor()
    ])

  def prepare_data(self):
    #print('PretextTaskDatamodule.prepare_data', area_ratio_patch)
    full_data, full_labels = generate_dataset(self.main_path,
                                              imsize=self.imsize,
                                              area_ratio_patch=self.area_ratio_patch,
                                              aspect_ratio_patch=self.aspect_ratio_patch,
                                              scar_width=self.scar_width,
                                              scar_thiccness=self.scar_thiccness,
                                              offset=self.augmentation_offset)
    
    full_data,full_labels = list2np(full_data, full_labels)
    full_data,full_labels = np2tensor(full_data, full_labels)

    full_train_data, test_data, full_train_labels, test_labels = train_test_split(
            full_data, full_labels, test_size=self.test_size, random_state=self.seed)
    
    train_data, val_data, train_labels, val_labels = train_test_split(
        full_train_data, 
        full_train_labels, 
        test_size=self.val_size, 
        random_state=self.seed
        )

    self.train_dataset = PretextTaskDataset(train_data, 
                                            train_labels, 
                                            transform=self.transform)

    self.val_dataset = PretextTaskDataset(val_data, 
                                          val_labels, 
                                          transform=self.transform)

    self.test_dataset = PretextTaskDataset(test_data, 
                                            test_labels, 
                                            transform=self.transform)
      

  def train_dataloader(self):
      return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

  def val_dataloader(self):
      return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=True)

  def test_dataloader(self):
      return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)

In [ ]:
class AnomalyDetectionDatamodule(pl.LightningDataModule):
  def __init__(self, main_path:str, 
               batch_size:int=64, 
               seed:int=0, 
               imsize:int=(256,256)):
    
    self.save_hyperparameters()
    self.prepare_data_per_node = False
    self.main_path = main_path
    self.batch_size = batch_size
    self.imsize = imsize
    self.seed = seed

    random.seed(seed)
    np.random.seed(seed)
    torch.random.manual_seed(seed)
    
    self.transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor()
    ])

  def prepare_data(self):
    full_data, full_labels = load_mvtec_dataset_imgs(self.main_path,
                                              imsize=self.imsize)
    
    full_data,full_labels = list2np(full_data, full_labels)
    full_data,full_labels = np2tensor(full_data, full_labels)

    self.test_dataset = PretextTaskDataset(full_data, 
                                            full_labels, 
                                            transform=self.transform)

  def test_dataloader(self):
      return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=True)

### Model

In [ ]:
class PretextModel(pl.LightningModule):
  def __init__(self, 
               num_classes=2, 
               seed=0):
    super().__init__()
    self.save_hyperparameters()
    self.seed = seed

    random.seed(seed)
    np.random.seed(seed)
    torch.random.manual_seed(seed)

    self.feature_extractor = resnet18(weights="IMAGENET1K_V1")

    self.feature_extractor.eval()
    for param in self.feature_extractor.parameters():
        param.requires_grad = False

    self.feature_extractor.fc = nn.Identity()
    self.classifier = nn.Sequential(
        nn.Linear(512, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(inplace=True),
        nn.Linear(512, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(inplace=True),
        nn.Linear(512, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(inplace=True),
        nn.Linear(512, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(inplace=True)
        )
    self.out = nn.Linear(128, num_classes)
      
  def compute_features(self, x):
      x = x.float()
      x = self.feature_extractor.conv1(x)
      x = self.feature_extractor.bn1(x)
      x = self.feature_extractor.relu(x)
      x = self.feature_extractor.maxpool(x)

      l1 = self.feature_extractor.layer1(x)
      l2 = self.feature_extractor.layer2(l1)
      l3 = self.feature_extractor.layer3(l2)
      l4 = self.feature_extractor.layer4(l3)

      x = self.feature_extractor.avgpool(l4)
      x = torch.flatten(x, 1)
      return (x, l1, l2, l3, l4)


  def forward(self, x):
    x = x.float()
    features = self.feature_extractor(x)
    features = features.view(features.size(0), -1)

    embeddings = self.classifier(features)
    output = self.out(embeddings)

    output = torch.sigmoid(output)
    return (output, embeddings)

In [ ]:
class ClassificationTask(pl.LightningModule):
    def __init__(self, num_classes=2, 
                 lr=0.03,
                 seed=0):
        super().__init__()
        self.lr=lr
        self.model = PretextModel(num_classes=num_classes)
    
    def forward(self, x):
        output, embeddings = self.model(x)
        return  output, embeddings

    def training_step(self, batch, batch_idx):    
        x,y = batch
        y_hat, _ = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        acc = accuracy(y_hat, y)

        metrics = {"train_accuracy": acc, "train_loss": loss}
        self.log_dict(metrics,
                      on_step=False,
                      on_epoch=True,
                      prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)

        metrics = {"val_accuracy": acc, "val_loss": loss}
        self.log_dict(metrics,
                      on_step=False,
                      on_epoch=True,
                      prog_bar=True)

        return metrics

    def test_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        
        metrics = {"test_accuracy": acc, "test_loss": loss}
        self.log_dict(metrics,
                      on_step=False,
                      on_epoch=True,
                      prog_bar=True)
        
        return metrics

    def _shared_eval_step(self, batch, batch_idx):
        x,y = batch
        y_hat, _ = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        acc = accuracy(y_hat, y)
        return loss, acc

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_hat, _ = self.model(x)
        return y_hat

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.model.parameters(), self.lr, momentum=0.9, weight_decay=0.00003)
        return optimizer

In [ ]:
class MetricTracker(pl.Callback):
  def __init__(self):
    
    self.log_metrics = {
        'train':{
            'accuracy':[],
            'loss':[]
        },
        'val':{
            'accuracy':[],
            'loss':[]
        }
    }

  def on_train_epoch_end(self, trainer, pl_module):
    elogs = trainer.logged_metrics
    self.log_metrics['train']['accuracy'].append(elogs['train_accuracy'].item())
    self.log_metrics['train']['loss'].append(elogs['train_loss'].item())
    self.log_metrics['val']['accuracy'].append(elogs['val_accuracy'].item())
    self.log_metrics['val']['loss'].append(elogs['val_loss'].item())

### Execution

Datamodule

In [ ]:
subject = 'tile'
main_path = '/content/dataset/'+subject+'/train/good/'

imsize = (256,256)
augmentation_offset = 0.1
area_ratio_patch = (0.02, 0.15)
aspect_ratio_patch = ((0.3, 1),(1, 3.3))
scar_width = (4,64)
scar_thiccness = (10,50)

batch_size_train = 64
batch_size_test = 8
train_test_split_val = 0.2
train_val_split_val = 0.2

seed = 0
lr = 0.003
epochs = 30
checkpoint_name = subject+'_best_model_weights.ckpt'
num_classes = 2

In [ ]:
datamodule = PretextTaskDatamodule(main_path, 
                                imsize=imsize, 
                                seed=seed,
                                batch_size=batch_size_train,
                                train_test_split=train_test_split_val,
                                train_val_split=train_val_split_val,
                                area_ratio_patch=area_ratio_patch,
                                aspect_ratio_patch=aspect_ratio_patch,
                                scar_width=scar_width,
                                scar_thiccness=scar_thiccness,
                                augmentation_offset=augmentation_offset)
datamodule.prepare_data()
print(datamodule.train_dataset.images.shape)
print(datamodule.val_dataset.images.shape)
print(datamodule.test_dataset.images.shape)

torch.Size([2355, 256, 256, 3])
torch.Size([589, 256, 256, 3])
torch.Size([736, 256, 256, 3])


In [ ]:
mvtec_test_data = AnomalyDetectionDatamodule(
    '/content/dataset/'+subject+'/test/',
    imsize=imsize, 
    batch_size=batch_size_test)
mvtec_test_data.prepare_data()
print(mvtec_test_data.test_dataset.images.shape)

torch.Size([117, 256, 256, 3])


In [ ]:
plt.hist(datamodule.train_dataset.labels)
plt.show()
plt.hist(datamodule.val_dataset.labels)
plt.show()
plt.hist(datamodule.test_dataset.labels)
plt.show()

Model summary

In [ ]:
from torchsummary  import summary
task = ClassificationTask()
summary(task.model.cuda(), (3,256,256))

In [ ]:
task = ClassificationTask()
print(task)

Training

In [ ]:
task = ClassificationTask(num_classes=num_classes,
                          lr=lr,
                          seed=seed)
cb = MetricTracker()
trainer = pl.Trainer(callbacks= [cb],
                     accelerator='auto', 
                     devices=1, 
                     max_epochs=epochs, 
                     check_val_every_n_epoch=1)

trainer.fit(task, datamodule=datamodule)
trainer.save_checkpoint(checkpoint_name)

Test

In [ ]:
task = ClassificationTask().load_from_checkpoint(checkpoint_name, model=task.model)
trainer.test(task, dataloaders=datamodule.test_dataloader())

In [ ]:
trainer.test(task, dataloaders=mvtec_test_data.test_dataloader())

In [ ]:
def report(trainer, task, test_imgs, labels):
  input = torch.permute(test_imgs, (0,3,1,2))

  preds, _ = task(input)
  #preds = trainer.predict(task, test_imgs)
  #print(preds)
  predicted = torch.max(preds.data, 1)
  preds = predicted.indices

  print(classification_report(preds, labels, labels=[0,1]))

In [ ]:
report(trainer, task, datamodule.test_dataset.images[:83],
       datamodule.test_dataset.labels[:83])
report(trainer, task, mvtec_test_data.test_dataset.images,
       mvtec_test_data.test_dataset.labels)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        42
           1       0.98      1.00      0.99        41

    accuracy                           0.99        83
   macro avg       0.99      0.99      0.99        83
weighted avg       0.99      0.99      0.99        83

              precision    recall  f1-score   support

           0       0.95      0.45      0.61        42
           1       0.63      0.98      0.77        41

    accuracy                           0.71        83
   macro avg       0.79      0.71      0.69        83
weighted avg       0.79      0.71      0.69        83



History track

In [ ]:
def plot_history(network_history, epochs):
    x_plot = list(range(1,epochs+1))
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(x_plot, network_history['train']['loss'])
    plt.plot(x_plot, network_history['val']['loss'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(x_plot, network_history['train']['accuracy'])
    plt.plot(x_plot, network_history['val']['accuracy'])
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()

plot_history(cb.log_metrics, epochs)

t-SNE visualization (cutpaste fine tuned)

In [ ]:
input = torch.permute(datamodule.test_dataset.images[:100], (0,3,1,2))
_, embeddings = task(input)
y = datamodule.test_dataset.labels[:100]
tsne = TSNE(n_components=2, random_state=0)
tsne_results = tsne.fit_transform(embeddings.detach().numpy())
tx = tsne_results[:, 0]
ty = tsne_results[:, 1]

df = pd.DataFrame()
df["labels"] = y
df["comp-1"] = tx
df["comp-2"] = ty
sns.scatterplot(hue=df.labels.tolist(),
                x='comp-1',
                y='comp-2',
                palette=sns.color_palette("hls", 2),
                data=df).set(title="Generated dataset projection") 

In [ ]:
input = torch.permute(mvtec_test_data.test_dataset.images[:100], (0,3,1,2))
_, embeddings = task(input)

y = mvtec_test_data.test_dataset.labels[:100]
tsne = TSNE(n_components=2, random_state=0)
tsne_results = tsne.fit_transform(embeddings.detach().numpy())
tx = tsne_results[:, 0]
ty = tsne_results[:, 1]

df = pd.DataFrame()
df["labels"] = y
df["comp-1"] = tx
df["comp-2"] = ty
sns.scatterplot(hue=df.labels.tolist(),
                x='comp-1',
                y='comp-2',
                palette=sns.color_palette("hls", 2),
                data=df).set(title="Generated dataset projection")

## Pipeline

### Pipeline function

In [ ]:
def report(trainer, task, test_imgs, labels):
  input = torch.permute(test_imgs, (0,3,1,2))

  preds, _ = task(input)
  #preds = trainer.predict(task, test_imgs)
  #print(preds)
  predicted = torch.max(preds.data, 1)
  preds = predicted.indices

  print(classification_report(preds, labels, labels=[0,1]))

def plot_history(network_history, epochs, saving_path=''):
    x_plot = list(range(1,epochs+1))
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(x_plot, network_history['train']['loss'])
    plt.plot(x_plot, network_history['val']['loss'])
    plt.legend(['Training', 'Validation'])
    plt.savefig(saving_path+'loss.png')

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(x_plot, network_history['train']['accuracy'])
    plt.plot(x_plot, network_history['val']['accuracy'])
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.savefig(saving_path+'accuracy.png')
    plt.show()

In [ ]:
def run_pipeline(subject):
  print('>>> running pipeline ('+subject.upper()+')')
  #parameters
  main_path = '/content/dataset/'+subject+'/train/good/'
  result_path = '/content/results/'+subject+'/'

  imsize = (256,256)
  augmentation_offset = 0.1
  area_ratio_patch = (0.02, 0.15)
  aspect_ratio_patch = ((0.3, 1),(1, 3.3))
  scar_width = (4,64)
  scar_thiccness = (10,50)

  batch_size_train = 64
  batch_size_test = 8
  train_test_split_val = 0.2
  train_val_split_val = 0.2

  seed = 0
  lr = 0.003
  epochs = 30
  checkpoint_name = subject+'_best_model_weights.ckpt'
  num_classes = 2

  #datasets
  print('>>> generating dataset')
  datamodule = PretextTaskDatamodule(main_path, 
                                imsize=imsize, 
                                seed=seed,
                                batch_size=batch_size_train,
                                train_test_split=train_test_split_val,
                                train_val_split=train_val_split_val,
                                area_ratio_patch=area_ratio_patch,
                                aspect_ratio_patch=aspect_ratio_patch,
                                scar_width=scar_width,
                                scar_thiccness=scar_thiccness,
                                augmentation_offset=augmentation_offset)
  datamodule.prepare_data()
  print(datamodule.train_dataset.images.shape)
  print(datamodule.val_dataset.images.shape)
  print(datamodule.test_dataset.images.shape)

  mvtec_test_data = AnomalyDetectionDatamodule(
    '/content/dataset/'+subject+'/test/',
    imsize=imsize, 
    batch_size=batch_size_test)
  mvtec_test_data.prepare_data()
  print(mvtec_test_data.test_dataset.images.shape)

  #model
  print('>>> initalizing model (pretrained resnet)')
  task = ClassificationTask(num_classes=num_classes,
                          lr=lr,
                          seed=seed)
  print('>>> defining callbacks')
  cb = MetricTracker()

  #training
  print('>>> start training')
  trainer = pl.Trainer(callbacks= [cb],
                      accelerator='auto', 
                      devices=1, 
                      max_epochs=epochs, 
                      check_val_every_n_epoch=1)

  trainer.fit(task, datamodule=datamodule)
  trainer.save_checkpoint(result_path+checkpoint_name)

  #test
  print('>>> start testing (artificial dataset)')
  task = ClassificationTask().load_from_checkpoint(result_path+checkpoint_name, model=task.model)
  trainer.test(task, dataloaders=datamodule.test_dataloader())
  print('>>> start testing (real defects)')
  trainer.test(task, dataloaders=mvtec_test_data.test_dataloader())

  #reports
  print('>>> metrics (artificial defects)')
  report(trainer, task, datamodule.test_dataset.images[:83],
       datamodule.test_dataset.labels[:83])
  print('>>> metrics (real defects)')
  report(trainer, task, mvtec_test_data.test_dataset.images,
       mvtec_test_data.test_dataset.labels)
  
  #history
  print('>>> training history')
  plot_history(cb.log_metrics, epochs, result_path)

  #t-sne
  print('>>> t-sne (artificial defects)')
  input = torch.permute(datamodule.test_dataset.images[:83], (0,3,1,2))
  _, embeddings = task(input)
  y = datamodule.test_dataset.labels[:83]
  tsne = TSNE(n_components=2, random_state=0)
  tsne_results = tsne.fit_transform(embeddings.detach().numpy())
  tx = tsne_results[:, 0]
  ty = tsne_results[:, 1]

  df = pd.DataFrame()
  df["labels"] = y
  df["comp-1"] = tx
  df["comp-2"] = ty
  plt.figure()
  sns.scatterplot(hue=df.labels.tolist(),
                  x='comp-1',
                  y='comp-2',
                  palette=sns.color_palette("hls", 2),
                  data=df).set(title="Generated dataset projection (cutpaste)") 
  plt.savefig(result_path+'tsne_artificial.png')

  print('>>> t-sne (real defects)')
  input = torch.permute(mvtec_test_data.test_dataset.images[:83], (0,3,1,2))
  _, embeddings = task(input)
  y = mvtec_test_data.test_dataset.labels[:83]
  tsne = TSNE(n_components=2, random_state=0)
  tsne_results = tsne.fit_transform(embeddings.detach().numpy())
  tx = tsne_results[:, 0]
  ty = tsne_results[:, 1]

  df = pd.DataFrame()
  df["labels"] = y
  df["comp-1"] = tx
  df["comp-2"] = ty
  plt.figure()
  sns.scatterplot(hue=df.labels.tolist(),
                  x='comp-1',
                  y='comp-2',
                  palette=sns.color_palette("hls", 2),
                  data=df).set(title="Generated dataset projection (cutpaste)") 
  plt.savefig(result_path+'tsne_artificial.png')

### Pipeline execution

In [ ]:
run_pipeline('tile')